In [ ]:
# colDate = 'Response date'
# def parse_dates(date):
#     if not isinstance(date, str):
#         # print(date)
#         date = str(date)
#         # print(date)
#         # Try parsing the date as dd-mm-yyyy
#         converted =  pd.to_datetime(date, format='mixed', errors='raise')
#         print('Already a date:', converted.strftime('%d-%m-%Y'))
#         return converted
#     else:
#         try:
#             # If it fails, try parsing as mm-dd-yyyy
#             converted = pd.to_datetime(date, format='%d/%m/%Y %I:%M:%S %p', errors='raise')
#             print('Converted:', converted.strftime('%d-%m-%Y'))
#             return converted
#         except:
#             # If both fail, return NaT
#             print('Cannot convert')
#             return date
# df_[colDate] = df_[colDate].apply(parse_dates)
# df_.to_excel('BOH Third Year - ORAL30002/2024 ORAL30002Clinical Assessment Form All.xlsx', index=False)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
from variableUtils import *
from Utils import * 
from pprint import pprint
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Table as RLTable, TableStyle
from reportlab.platypus import Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from PyPDF2 import PdfReader, PdfWriter
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


(841.68, 1190.8799999999999)


In [ ]:
folderpath = 'BOH3\Pebble Pad'
# read all the files in the folder
files = os.listdir(folderpath)
dfListDict = {}
# dfList =[]
colusername = 'Username'
usernamelist = ['nmusis', 'jkfry', 'up']

for file in files:
    ext = os.path.splitext(file)[1]
    if ext == '.xlsx':
        df = pd.read_excel(f'{folderpath}/{file}')
    elif ext == '.csv':
        df = pd.read_csv(f'{folderpath}/{file}')
    else:
        continue
    # second last column rename as Overall Competency
    df.rename(columns={df.columns[-2]: 'Overall Competency'}, inplace=True)
    # second column rename as Student ID
    # df.rename(columns={df.columns[1]: 'Student ID'}, inplace=True)
    # find the column that has Student name in it
    for col in df.columns:
        if 'Student name' in col:
            df.rename(columns={col: 'Student Name'}, inplace=True)
            colName = 'Student Name'
        elif 'session date' in col.strip().lower():
            df.rename(columns={col: 'Session Date'}, inplace=True)
            colDate = 'Session Date'
    
    usernames = df[colusername].unique()
    for username in usernames:
            if username in dfListDict.keys():
                dfListDict[username].append(df[df[colusername] == username])
            else:
                dfListDict[username] = [df[df[colusername] == username]]


pprint(dfListDict)

In [ ]:
pathref = 'BOH3\\2024 ORAL30002Clinical Assessment Form All.xlsx'
folderpath = 'BOH3\Pebble Pad'
dfid = pd.read_excel(pathref)
pprint(dfid.columns)
# create a mapping dict of Username to ID
usernameCol = 'Username'
idCol = 'Student ID'

dfid = dfid[[usernameCol, idCol]]
dfid = dfid.drop_duplicates()
dfid = dfid.dropna()
dfid = dfid.set_index(usernameCol)
dfid[idCol] = dfid[idCol].astype(int)
dfid.to_excel(f'{folderpath}/ID Mapping.xlsx')
dfid = dfid.to_dict()[idCol]
display(dfid)

# read files
# df1 = pd.read_excel(f'BOH3\Pebble Pad\\2024 ORAL30002 Rotation 4 Clinical Assessment Form.xlsx')
df2 = pd.read_csv(f'BOH3\Pebble Pad\\2024 ORAL30002 Rotation 7 Clinical Assessment Form.csv')
# df1[idCol] = df1[colusername].map(dfid)
df2[idCol] = df2[colusername].map(dfid)
# df1.to_excel(f'BOH3\Pebble Pad\\2024 ORAL30002 Rotation 4 Clinical Assessment Form.xlsx', index=False)
df2.to_excel(f'BOH3\Pebble Pad\\2024 ORAL30002 Rotation 7 Clinical Assessment Form.xlsx', index=False)

# fill the ID column if not exist
for username, dfList in dfListDict.items():
    for df in dfList:
        if idCol not in df.columns:
            df[idCol] = df[colusername].map(dfid)
            display(df)
# for df in dfList:
#     if idCol not in df.columns:
#         df[idCol] = df[colusername].map(dfid)
#         display(df)

In [ ]:
folderpath = 'BOH3\Pebble Pad'
# read all the files in the folder
files = os.listdir(folderpath)
dfListDict = {}
for file in files:
    ext = os.path.splitext(file)[1]
    if ext == '.xlsx':
        df = pd.read_excel(f'{folderpath}/{file}')
    elif ext == '.csv':
        df = pd.read_csv(f'{folderpath}/{file}')
    else:
        continue
    # eXtract Rotation \d
    rotation = re.findall(r'Rotation \d', file)
    if len(rotation) == 0:
        continue
    else:
        rotation = rotation[0]
    print(rotation)

    dfListDict[rotation] = df

pprint(dfListDict)
combinedDf = pd.concat(dfListDict.values(), ignore_index=True)
combinedDf.to_excel(f'{folderpath}\Combined.xlsx', index=False)

## With Paeds Data do analysis

In [44]:


def combineAges(df, ageCols):
    # Combine all age columns into a single series, ignoring NaN values
    combinedAges = pd.concat([df[col] for col in ageCols], ignore_index=True).dropna()
    # Convert ages to numeric, forcing errors to NaN and drop invalid entries
    combinedAges = pd.to_numeric(combinedAges, errors='coerce').dropna()
    # remove ages greater than 17
    combinedAges = combinedAges[combinedAges.lt(18)]
    return combinedAges


def createAgeHist(ageDf, ageCols, studentID=None, studentName=None):
    ageDf = ageDf.astype('Int64')
    # Combine all age columns into a single series, ignoring NaN values
    combinedAges = combineAges(ageDf, ageCols)
    # display(combinedAges)   
    print(len(combinedAges))
    # Plotting the distribution of patient ages
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
    
    # Define bins with a step of 1 (to create every 2 bins)
    bins = np.arange(0, 18, 1)
    
    # Plot the histogram
    ax.hist(combinedAges, bins=bins, edgecolor='black', alpha=0.5)
    
    # Set the title
    title = f'Distribution of Patient Ages'
    if studentID:
        title += f' for {studentName if studentName else ""} {studentID}'
    ax.set_title(title)
    
    # Set x-axis labels and ticks
    ax.set_xlabel('Age')
    ax.set_xticks(bins)
    
    # Ensure y-axis ticks are integers
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    ax.set_ylabel('Frequency')
    
    # Add gridlines with some transparency
    ax.grid(True, alpha=0.5)
    
    # Return the figure object
    return fig

In [57]:
df = pd.read_excel('BOH3\Pebble Pad\Combined.xlsx')
studentdf = pd.read_excel('2024 MDS Student List_v10.xlsx')
studentdf = studentdf[studentdf['Cohort'] =='BOH3 (2024)']
allstudents = studentdf[colId].unique()
print(len(allstudents))
thisstudents = df[colId].unique()
# find the student that is not in the list
for student in allstudents:
    if student not in thisstudents:
        print(student, studentdf[studentdf[colId] == student]['First Name'].values[0] + ' ' + studentdf[studentdf[colId] == student]['Surname'].values[0])


33
1180109 Paloma Araos-West
1255040 Zephaniah Josephene
1270550 Thuvasaki Mahinthan


In [ ]:
df = pd.read_excel('BOH3\Pebble Pad\Combined Paeds.xlsx')

In [59]:
df = pd.read_excel('BOH3\Pebble Pad\Combined Paeds.xlsx')
print(len(df[colId].unique()))
# Find the columns that have the age 
ageCols = [col for col in df.columns if 'patient age' in col.strip().lower()]
print(ageCols)
folderpath, file, ext = getFolderandFileName('BOH3\Pebble Pad\Combined Paeds.xlsx')
# create a pdf file with the same name as the excel file
pdfpath = f'{folderpath}\\{file}.pdf'
allIds = df[colId].unique()
elements = []
heading = Paragraph(f'Report on Paediatric Patients', headingStyle)
elements.append(KeepTogether([heading, Spacer(1, 0.5 * inch)]))
ageDf = df[ageCols]
colName = 'Student Name'


# Create a histogram of patient ages
fig = createAgeHist(ageDf, ageCols)
image = addPlotImage(fig)
elements.append(image)
elements.append(Spacer(1, 0.25 * inch))  



# for each student count number of Paeds
countDf = pd.DataFrame(columns=[colId, '# of Paeds Patients'])
# display(df.head())
for studentID in allIds:
    studentDf = df[df[colId] == studentID]
    # display(studentDf)
    combined = combineAges(studentDf, ageCols)
    nPatients = len(combined)
    # print(studentID, nPatients)
    countDf = pd.concat([countDf, pd.DataFrame([[studentID, nPatients]], columns=[colId, '# of Paeds Patients'])])
    # display(countDf)
    # ageDf = ageDf.set_index(colId)
countDf.sort_values('# of Paeds Patients', ascending=False, inplace=True)
# anonymize the student ID
countDf[colId] = [f'Student {i}' for i in range(1, len(countDf) + 1)]
countDf.set_index(colId, inplace=True)
countDf.to_excel(f'{folderpath}\\Number of Paeds Patients.xlsx')
# Create a bar chart
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
countDf.plot(kind='bar', ax=ax, legend=False, color='skyblue', edgecolor='black')
ax.set_title('Number of Paediatric Patients Seen')
ax.set_xlabel('Student ID')
ax.set_ylabel('Number of Patients')
for p in ax.patches:
    ax.annotate(
        f'{p.get_height()}', 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom', 
        fontsize=8
    )
ax.grid(True, alpha=0.5)
image = addPlotImage(fig)
elements.append(image)
elements.append(PageBreak())


# get number of average patients seen by each student between ages 0-5 6-11 and 12-17
ageGroups = [(0, 5), (6, 11), (12, 17)]
avgDf = pd.DataFrame(columns=['Age Group', 'Average # of Patients'])
combinedAges = combineAges(df, ageCols)
for group in ageGroups:
    lower, upper = group
    avg = len(combinedAges[(combinedAges >= lower) & (combinedAges <= upper)]) / len(allIds)
    avgDf = pd.concat([avgDf, pd.DataFrame([[f'{lower}-{upper}', int(avg)]], columns=['Age Group', 'Average # of Patients'])])

avgDf.set_index('Age Group', inplace=True)
display(avgDf)
# bar chart # show labels on top of bars
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
avgDf.plot(kind='bar', ax=ax, legend=False, color='skyblue', edgecolor='black')
ax.set_title('Average Number of Patients Seen by Age Group')
ax.set_xlabel('Age Group')
# Adding values on top of bars
for p in ax.patches:
    ax.annotate(
        f'{p.get_height()}', 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom'
    )
ax.set_ylim(0, avgDf['Average # of Patients'].max() + 5)    
ax.set_ylabel('Average Number of Patients')
ax.set_xticklabels(avgDf.index, rotation=0)
ax.grid(True, alpha=0.5)
image = addPlotImage(fig)
elements.append(image)
elements.append(Spacer(1, 0.25 * inch))

# get averages by clinic type
colClinic = 'Dental clinic/organization'
clinics = df[colClinic].unique()
clinics = [clinic for clinic in clinics if not pd.isna(clinic)]
clinicDf = pd.DataFrame(columns=[colClinic, 'Average # of Patients'])
for clinic in clinics:
    thisDf = df[df[colClinic] == clinic]
    print(clinic, len(thisDf))
    thisallIds = thisDf[colId].unique()
    combinedAges = combineAges(thisDf, ageCols)
    avg = len(combinedAges) / len(thisallIds)
    clinicDf = pd.concat([clinicDf, pd.DataFrame([[clinic, int(avg)]], columns=[colClinic, 'Average # of Patients'])])

clinicDf.set_index(colClinic, inplace=True)
clinicDf.sort_values('Average # of Patients', ascending=False, inplace=True)
display(clinicDf)
# bar chart # show labels on top of bars
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
clinicDf.plot(kind='bar', ax=ax, legend=False, color='skyblue', edgecolor='black')
ax.set_title('Average Number of Paeds Patients Seen by Clinic')
ax.set_xlabel('Clinic')
ax.set_ylim(0, clinicDf['Average # of Patients'].max() + 5)
# Adding values on top of bars
for p in ax.patches:
    ax.annotate(
        f'{p.get_height()}', 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom'
    )
ax.set_ylabel('Average Number of Patients')
ax.grid(True, alpha=0.5)
image = addPlotImage(fig)
elements.append(image)
elements.append(PageBreak())



# for each age group, get the items
elements.append(Paragraph('Top Procedures Done for Patients', headingStyle))
elements.append(Spacer(1, 0.5 * inch))
avgDfDict = {}
for lower, upper in ageGroups:
    # filter out rows that have age within the range for any of the age columns
    thisDf = df[df[ageCols].lt(upper).any(axis=1) & df[ageCols].ge(lower).any(axis=1)]
    filteredDf, procedure_columns, additional_procedure_columns = renameProcedureCols(thisDf)
    procedures = []
    for i, row in filteredDf.iterrows():
        procedures += getProcedures(row, procedure_columns, additional_procedure_columns)
    # get the count of each procedure
    procedureCounts = pd.Series(procedures).value_counts()
    # keep top 10 procedures
    procedureCounts = procedureCounts.head(30)
    # plot the bar chart
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
    procedureCounts.plot(kind='bar', ax=ax, legend=False, color='skyblue', edgecolor='black')
    ax.set_title(f'Top Procedures Done for Patients Aged {lower}-{upper}')
    ax.set_xlabel('Procedure Code')
    ax.set_ylabel('Frequency')
    for p in ax.patches:
        ax.annotate(
            f'{p.get_height()}', 
            (p.get_x() + p.get_width() / 2, p.get_height()), 
            ha='center', 
            va='bottom',
            fontsize=6
        )
    
    # show the total number of procedures done at top right corner
    ax.text(0.99, 0.99, f'Total: {procedureCounts.sum()}', transform=ax.transAxes, ha='right', va='top')
    ax.grid(True, alpha=0.5)
    image = addPlotImage(fig)
    elements.append(image)
    elements.append(Spacer(1, 0.25 * inch))

    avgDf = pd.DataFrame(columns=['Competency', 'Average Score out of 5'])
    # get average of each competency column
    competency_columns_supervisor = ['History, Examination & Investigations (Row 1).1',	'Diagnosis/es Problem List (Row 2).1',	
                                    'Treatment and Management Plan (Row 3).1',	'Appropriate patient care and communication (Row 4).1',	'Clinical and technical proficiency (Row 5).1',	
                                    'Applied Scientific and clinical knowledge base (Row 6).1',	'Professional behaviour and ethical conduct (Row 7).1']
    for col in competency_columns_supervisor:
        filteredDf[col] = filteredDf[col].astype('Int64')
        avg = filteredDf[col].mean()
        avg = round(avg, 2)
        avgDf = pd.concat([avgDf, pd.DataFrame([[col.split(' (')[0], avg]], columns=['Competency', 'Average Score out of 5'])])
    # avgDf.set_index('Competency', inplace=True)
    avgDfDict[(lower, upper)] = avgDf
    table = createTable(avgDf, 'Average Performance Scores', [5, 2], 0.9, [0, 1])
    elements.append(table)
    elements.append(Spacer(1, 0.25 * inch))
    display(avgDf)

doc = SimpleDocTemplate(pdfpath, pagesize=pageSize, leftMargin=leftMargin, 
                        rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
doc.build(elements)

28
['Patient information//Row 1//Patient Age', 'Patient information//Row 2//Patient Age', 'Patient information//Row 3//Patient Age', 'Patient information//Row 4//Patient Age', 'Patient information//Row 5//Patient Age', 'Patient information//Row 6//Patient Age', 'Patient information//Row 7//Patient Age']
2555


,Average # of Patients
Age Group,
0-5,18
6-11,48
12-17,24


IPC Health 371
EACH 104
La Trobe - Moe 329
Cohealth Footscray 116
GVH 71
Rumbalara OHC 47
RDHM 4
MDC 13
Not answered 103
YOUR Community Health 35


,Average # of Patients
Dental clinic/organization,
IPC Health,55
La Trobe - Moe,26
EACH,22
Cohealth Footscray,17
GVH,12
Not answered,10
Rumbalara OHC,5
YOUR Community Health,4
RDHM,1


Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Res

,Competency,Average Score out of 5
0,"History, Examination & Investigations",3.37
0,Diagnosis/es Problem List,3.28
0,Treatment and Management Plan,3.21
0,Appropriate patient care and communication,3.68
0,Clinical and technical proficiency,3.13
0,Applied Scientific and clinical knowledge base,3.36
0,Professional behaviour and ethical conduct,3.91


Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Res

,Competency,Average Score out of 5
0,"History, Examination & Investigations",3.45
0,Diagnosis/es Problem List,3.32
0,Treatment and Management Plan,3.26
0,Appropriate patient care and communication,3.65
0,Clinical and technical proficiency,3.15
0,Applied Scientific and clinical knowledge base,3.39
0,Professional behaviour and ethical conduct,3.89


Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Examination and diagnostic procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Periodontal and preventive procedures
Found: Res

,Competency,Average Score out of 5
0,"History, Examination & Investigations",3.48
0,Diagnosis/es Problem List,3.35
0,Treatment and Management Plan,3.33
0,Appropriate patient care and communication,3.69
0,Clinical and technical proficiency,3.17
0,Applied Scientific and clinical knowledge base,3.43
0,Professional behaviour and ethical conduct,3.89


In [79]:
df = pd.read_excel('BOH2\CAF+v0.1_October152024_19.01\CAF v0.1_October 15, 2024_19.01 Paeds filtered.xlsx')
# Find the columns that have the age 
ageCols = [colAge]
folderpath, file, ext = getFolderandFileName('BOH2\CAF+v0.1_October152024_19.01\CAF v0.1_October 15, 2024_19.01 Paeds filtered.xlsx')
# create a pdf file with the same name as the excel file
pdfpath = f'{folderpath}\\Paeds Report BOH2.pdf'
allIds = df[colId].unique()
elements = []
heading = Paragraph(f'Report on Paediatric Patients', headingStyle)
elements.append(KeepTogether([heading, Spacer(1, 0.5 * inch)]))
ageDf = df[ageCols]
colName = 'Student Name'

# Create a histogram of patient ages
display(ageDf.head())
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
bins = np.arange(0, 18, 1)
ax.hist(ageDf[colAge], bins=bins, edgecolor='black', alpha=0.5)
title = f'Distribution of Patient Ages'
ax.set_title(title)
ax.set_xlabel('Age')
ax.set_xticks(bins)
ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.set_ylabel('Frequency')
ax.grid(True, alpha=0.5, linestyle='--')
# show total number of patients at top right corner
ax.text(0.99, 0.99, f'Total: {len(ageDf)}', transform=ax.transAxes, ha='right', va='top')
image = addPlotImage(fig)
elements.append(image)
elements.append(Spacer(1, 0.25 * inch))  



# for each student count number of Paeds
countDf = pd.DataFrame(columns=[colId, '# of Paeds Patients'])
# display(df.head())
for studentID in allIds:
    studentDf = df[df[colId] == studentID]
    # display(studentDf)
    combined = combineAges(studentDf, ageCols)
    nPatients = len(combined)
    # print(studentID, nPatients)
    countDf = pd.concat([countDf, pd.DataFrame([[studentID, nPatients]], columns=[colId, '# of Paeds Patients'])])
    # display(countDf)
    # ageDf = ageDf.set_index(colId)
countDf.sort_values('# of Paeds Patients', ascending=False, inplace=True)
# anonymize the student ID
countDf[colId] = [f'Student {i}' for i in range(1, len(countDf) + 1)]
countDf.set_index(colId, inplace=True)
countDf.to_excel(f'{folderpath}\\Number of Paeds Patients.xlsx')
# Create a bar chart
# Split the data into two halves
half_index = len(countDf) // 2
countDf_first_half = countDf.iloc[:half_index]
countDf_second_half = countDf.iloc[half_index:]

# Create two vertical subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(figSize[0], figSize[1]/2))

# Plot the first half of the data
countDf_first_half.plot(kind='bar', ax=ax1, legend=False, color='skyblue', edgecolor='black')
ax1.set_title('Number of Paediatric Patients Seen')
ax1.set_xlabel(None)
ax1.set_xticklabels(countDf_first_half.index, rotation=90, fontsize=6)
ax1.set_ylabel('Number of Patients')
ax1.set_ylim(0, countDf_first_half['# of Paeds Patients'].max() + 5)
for p in ax1.patches:
    ax1.annotate(
        f'{p.get_height()}', 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom'
    )
ax1.grid(True, alpha=0.5, linestyle='--', axis='y')

# Plot the second half of the data
countDf_second_half.plot(kind='bar', ax=ax2, legend=False, color='skyblue', edgecolor='black')
ax2.set_xlabel('Student ID')
ax2.set_xticklabels(countDf_second_half.index, rotation=90, fontsize=6)
ax2.set_ylabel('Number of Patients')
ax2.set_ylim(0, countDf_second_half['# of Paeds Patients'].max() + 5)
ax2.grid(True, alpha=0.5, linestyle='--', axis='y')
for p in ax2.patches:
    ax2.annotate(
        f'{p.get_height()}', 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom'
    )
# Adjust layout for clarity
plt.tight_layout()

image = addPlotImage(fig)
elements.append(image)
elements.append(PageBreak())
import math
# get number of average patients seen by each student between ages 0-5 6-11 and 12-17
ageGroups = [(0, 5), (6, 11), (12, 17)]
avgDf = pd.DataFrame(columns=['Age Group', 'Average # of Patients'])
combinedAges = combineAges(df, ageCols)
for group in ageGroups:
    lower, upper = group
    avg = len(combinedAges[(combinedAges >= lower) & (combinedAges <= upper)]) / len(allIds)
    avgDf = pd.concat([avgDf, pd.DataFrame([[f'{lower}-{upper}', math.ceil(avg)]], columns=['Age Group', 'Average # of Patients'])])

avgDf.set_index('Age Group', inplace=True)
display(avgDf)
# bar chart # show labels on top of bars
fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
avgDf.plot(kind='bar', ax=ax, legend=False, color='skyblue', edgecolor='black')
ax.set_title('Average Number of Patients Seen by Age Group')
ax.set_xlabel('Age Group')
ax.set_xticklabels(avgDf.index, rotation=0)
# Adding values on top of bars
for p in ax.patches:
    ax.annotate(
        f'{p.get_height()}', 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom'
    )
ax.set_ylim(0, avgDf['Average # of Patients'].max() + 5)    
ax.set_ylabel('Average Number of Patients')
ax.grid(True, alpha=0.5)
image = addPlotImage(fig)
elements.append(image)
elements.append(Spacer(1, 0.25 * inch))
elements.append(PageBreak())
# # for each age group, get the items
elements.append(Paragraph('Top Procedures Done for Patients', headingStyle))
elements.append(Spacer(1, 0.5 * inch))
workbook, _, _ = loadWorkbook('BOH2\CAF+v0.1_October152024_19.01\CAF v0.1_October 15, 2024_19.01 Paeds guttman.xlsx')
countDf = pd.DataFrame(columns=['Item', 'Age Group', 'Count'])
for item in workbook.sheetnames:
    df2 = loadDfFromSheet(workbook, item)
    code = item.split('_')[0]
    # get counts for each age group
    for lower, upper in ageGroups:
        thisDf = df2[df2[colAge].ge(lower) & df2[colAge].le(upper)]
        count = len(thisDf)
        countDf = pd.concat([countDf, pd.DataFrame([[code, f'{lower}-{upper}', count]], columns=['Item', 'Age Group', 'Count'])])
display(countDf)
# aggregate over item adn age group
countDf = countDf.groupby(['Item', 'Age Group']).sum().reset_index()
# createa  pivot table with item as index and age group as columns and count as values
pivot = countDf.pivot(index='Item', columns='Age Group', values='Count')
pivot.fillna(0, inplace=True)
pivot = pivot.astype(int)
display(pivot)
# create three bar charts each for an age group
for lower, upper in ageGroups:
    thisDf = pivot[f'{lower}-{upper}']
    # remove items with 0 count and sort thisDf is a series
    
    thisDf = thisDf[thisDf.gt(0)].sort_values(ascending=False)
    fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 3))
    thisDf.plot(kind='bar', ax=ax, legend=False, color='skyblue', edgecolor='black')
    ax.set_title(f'Procedures Done for Patients Aged {lower}-{upper}')
    ax.set_xlabel('Procedure Code')
    ax.set_ylabel('Frequency')
    for p in ax.patches:
        ax.annotate(
            f'{p.get_height()}', 
            (p.get_x() + p.get_width() / 2, p.get_height()), 
            ha='center', 
            va='bottom',
            fontsize=9
        )
    ax.grid(True, alpha=0.5, linestyle='--')
    # show total number of procedures at top right corner
    ax.text(0.99, 0.99, f'Total: {thisDf.sum()}', transform=ax.transAxes, ha='right', va='top')
    image = addPlotImage(fig)
    elements.append(image)
    elements.append(Spacer(1, 0.25 * inch))
doc = SimpleDocTemplate(pdfpath, pagesize=pageSize, leftMargin=leftMargin, 
                        rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
doc.build(elements)

,Patient Age
0,8.0
1,8.0
2,10.0
3,10.0
4,1.0


,Average # of Patients
Age Group,
0-5,3
6-11,12
12-17,1


('Loaded workbook: BOH2\\CAF+v0.1_October152024_19.01 | CAF v0.1_October 15, '
 '2024_19.01 Paeds guttman | .xlsx')
("Workbook sheets: ['011_COE', '012_POE', '013_LIMITED OE', '022_I-O RAD', "
 "'111_CLINIC', '114_CLINIC', '115_CLINIC', '121_CLINIC REMIN', '121_SIM "
 "REMIN', '123_CLINIC', '131_CLINIC', '141_CLINIC', '161_CLINIC', '161_SIM', "
 "'221_CLINIC', '311_CLINIC']")


,Item,Age Group,Count
0,011,0-5,69
0,011,6-11,372
0,011,12-17,24
0,012,0-5,0
0,012,6-11,9
0,012,12-17,1
0,013,0-5,2
0,013,6-11,2
0,013,12-17,1
0,022,0-5,22


Age Group,0-5,12-17,6-11
Item,,,
011,69,24,372
012,0,1,9
013,2,1,2
022,22,7,142
111,2,2,20
114,7,4,62
115,0,1,1
121,37,14,237
123,0,0,2


In [ ]:
# get total age distribution
df = pd.read_excel('BOH3\Pebble Pad\Combined Paeds.xlsx')
folderpath, file, ext = getFolderandFileName('BOH3\Pebble Pad\Combined Paeds.xlsx')
def createReport(df, studentId = None, studentName=None):
    ageCols = [col for col in df.columns if 'patient age' in col.strip().lower()]
    ageDf = df[ageCols]
    colClinic = 'Dental clinic/organization'
    filename = f'{folderpath}\BOH3 Paeds.pdf' if studentId is None \
          else f'{folderpath}\Student Paeds\{studentId}{f" {studentName}" if studentName is not None else ""}.pdf'
    doc = SimpleDocTemplate(f'{filename}', pagesize=pageSize)
    elements = []


    fig = createAgeHist(ageDf, studentID='All Clinics')
    image = addPlotImage(fig, 0.7)
    # header
    heading = 'Paeds Report for BOH3' if studentId is None else f'{studentId} ({studentName})'
    elements.append(Paragraph(heading, headingStyle))
    elements.append(Spacer(1, 24))
    elements.append(image)
    # elements.append(PageBreak())
    # get total age distribution for each clinic
    clinics = df[colClinic].unique()
    # remove na
    clinics = clinics[~pd.isna(clinics)]
    print(clinics)
    for clinic in clinics:
        dfClinic = df[df[colClinic] == clinic]
        ageDf = dfClinic[ageCols]
        fig = createAgeHist(ageDf, studentID=clinic)
        image = addPlotImage(fig, 0.7)
        merged = KeepTogether([Paragraph(clinic, subheadingStyle), Spacer(1, 12), image])
        elements.append(merged)



    # get number of patients for each clinic and plot a bar chart meaning counting all non NaN values in patient id columns
    colPatientID = [col for col in df.columns if 'patient id' in col.strip().lower()]
    print(colPatientID)
    patientcount = {}
    for i, row in df.iterrows():
        for col in colPatientID:
            ageColumn = '//'.join(col.split('//')[0:2]) + '//Patient Age'
            if row[ageColumn] >= 18:
                continue
            if pd.notna(row[col]):
                if row[colClinic] in patientcount.keys():
                    patientcount[row[colClinic]] += 1
                else:
                    patientcount[row[colClinic]] = 1

    # remove na keys
    patientcount = {k: v for k, v in patientcount.items() if pd.notna(k)}
    print(patientcount)
    # plt bar chart of patient count
    # Assuming patientcount is already available as a dictionary
    patientcountDf = pd.DataFrame(patientcount.items(), columns=['Clinic', 'Number of Patients'])
    patientcountDf.sort_values(by='Number of Patients', ascending=False, inplace=True)

    # Display sorted DataFrame
    display(patientcountDf)

    # Create a horizontal bar chart
    def wrap_label(label):
        words = label.split()
        if len(words) > 3:
            # Insert a newline after the third word
            return ' '.join(words[:3]) + '\n' + ' '.join(words[3:])
        return label

    # Apply the label wrapping function only for labels with more than 3 words
    wrapped_labels = [wrap_label(label) for label in patientcountDf['Clinic']]

    # Create a horizontal bar chart with the modified labels
    plt.figure(figsize=(10, 6))
    plt.barh(patientcountDf['Clinic'], patientcountDf['Number of Patients'], color='skyblue')

    # Set the title and labels
    plt.title('Number of Paeds Patients for each Clinic')
    plt.xlabel('Number of Patients')
    plt.ylabel('Clinic')

    # Add gridlines to the x-axis (horizontal chart)
    plt.grid(axis='x', alpha=0.5)

    # Apply the wrapped labels
    plt.yticks(ticks=range(len(wrapped_labels)), labels=wrapped_labels)

    # Add the plot to the report image
    image = addPlotImage(plt.gcf()) 
    elements.insert(2, KeepTogether([image, Spacer(1, 16)]))
    elements.insert(4, PageBreak()) 

    doc.build(elements)

idCol = 'Student ID'
ids = df[idCol].unique()
for id in ids:
    dfid = df[df[idCol] == id]
    name = dfid['Student Name'].iloc[0]
    createReport(dfid, studentId=id, studentName=name)

In [ ]:

# df_1 = pd.read_excel('BOH Third Year - ORAL30002/2024 ORAL30002 Rotation 1 Clinical Assessment Form 2.xlsx')
# df_2 = pd.read_excel('BOH Third Year - ORAL30002/2024 ORAL30002 Rotation 2 Clinical Assessment Form 2.xlsx')
# df_3 = pd.read_excel('BOH Third Year - ORAL30002/2024 ORAL30002 Rotation 3 Clinical Assessment Form 2.xlsx')
# df_4 = pd.read_excel('BOH Third Year - ORAL30002/2024 ORAL30002 Rotation 4 Clinical Assessment Form 2.xlsx')

# df_ = pd.concat([df_1, df_2, df_3, df_4], axis=0, ignore_index=True)
filepath = 'DDS3\Fix Pros\DENT90120 Prosthodontics Pebble Pad.xlsx'
filepath = 'BOH3\\2024 ORAL30002 Rotation 6 Clinical Assessment Form.csv'
filepath = 'BOH3\Pebble Pad\\2024 ORAL30002 Rotation 7 Clinical Assessment Form.xlsx'
folder, file, ext = getFolderandFileName(filepath)
foldersub = file.split(' ')[2:min(4, len(file.split(' ')))]
foldersub = ' '.join(foldersub)
output_dir = f'{folder}/{foldersub} pdfs'
os.makedirs(output_dir, exist_ok=True)
df_ = pd.read_excel(filepath)
# create a colID column based on the username
# df_[colId] = df_[usernameCol].map(dfid)
# df_.to_csv(f'{folder}/{file} filled.{ext}', index=False)
display(df_.head())

In [ ]:
df = df_.copy()
colId = 'Student ID'
colDate = 'Session Date'

df[colDate] = pd.to_datetime(df[colDate], format='mixed', errors='coerce').dt.date
# df.rename({'Unnamed: 1': 'Student ID'}, axis=1, inplace=True)
# extract just the date as a string
df[colDate] = df[colDate].astype(str)
df[colId] = df[colId].astype(int, errors='ignore')
colName = 'Respondent'

# df[colDate] = pd.to_datetime(df[colDate], format='%d %b %Y').dt.date

i=0
for col in df.columns:
    print(i, col)
    i+=1
print(df.shape)
df.head()

In [5]:

# Function to extract numbers from column names using regex
def extract_numbers(column_name, procedure_columns):
    match = re.search(r'\d+', column_name)
    if match:
        procedure_columns.append(match.group())
    return (match.group() if match else column_name, procedure_columns)

# extract the procedure columns
# procedure_columns = [col for col in df.columns if 'Examination and diagnostic procedures' in col
#                      or 'Periodontal and preventive procedures' in col
#                      or 'Restorative procedures' in col
#                      or 'Extractions and endodontic/pulp therapy' in col]
competency_columnsDict = {}
additional_procedure_columnsDict = {}
procedure_columnsDict = {}
competency_columns_supervisorDict = {}
competency_columns_studentDict = {}
def renameProcedureCols(df_):
    df = df_.copy()
    additional_procedure_columns = [col for col in df.columns if 'Additional Procedures' in col]
    procedure_columns = []
    procedure_patterns = [
        'Examination and diagnostic procedures', 
        'Periodontal and preventive procedures',
        'Restorative procedures', 
        'Extractions and endodontic/pulp therapy',
        'Prosthodontics'
    ]

    # Iterate through each column and rename if it matches any pattern

    for col in df.columns:
        # print(col)
        for pattern in procedure_patterns:
            if pattern in col:
                print('Found:', pattern)
                # Extract numbers and rename the column
                new_col_name , procedure_columns = extract_numbers(col, procedure_columns)
                df.rename(columns={col: new_col_name}, inplace=True)
    # Rename columns based on extracted numbers
    # df.rename(columns=lambda col: extract_numbers(col, procedure_columns) if 'Examination and diagnostic procedures' in col else col, inplace=True)
    # df.rename(columns=lambda col: extract_numbers(col, procedure_columns) if 'Periodontal and preventive procedures' in col else col, inplace=True)
    # df.rename(columns=lambda col: extract_numbers(col, procedure_columns) if 'Restorative procedures' in col else col, inplace=True)
    # df.rename(columns=lambda col: extract_numbers(col, procedure_columns) if 'Extractions and endodontic/pulp therapy' in col else col, inplace=True)
    # df.rename(columns=lambda col: extract_numbers(col, procedure_columns) if 'Prosthodontics' in col else col, inplace=True)
    # print(procedure_columns)
    df.rename({'Unnamed: 105': 'Overall Competency'}, axis=1, inplace=True)
    rename_values = {'Able to perform task/s independently': 4, 'Able to perform task/s under my indirect supervision': 3, 
                    'Able to perform task/s under my direct/proactive supervision': 2, 'Not yet able to perform task/s': 1, 
                    'Not yet able to perform task/s.': 1, 'Not yet able to perform task/s. Critical concern':1, 'Not observed': np.nan,
                    'Competent/Independent': 4, 'Developing competency/Indirect supervision required': 3, 'Dependent/Close supervision required': 2, 
                    'Not yet competent/Critical concerns': 1, 'I am able to perform this task independently': 4, 
                    'I am able to perform this task with indirect/limited supervision': 3, 
                    'I am able to perform this task with direct/close supervision': 2, 'I am not yet able to perform this task': 1, 
                    'I am not yet able to perform this task.': 1, 'I did not have to perform that task today': np.nan,
                    'I did not have to perform that task today.': np.nan}

    competency_columns_supervisor = ['History, Examination & Investigations (Row 1).1',	'Diagnosis/es Problem List (Row 2).1',	'Treatment and Management Plan (Row 3).1',	'Appropriate patient care and communication (Row 4).1',	'Clinical and technical proficiency (Row 5).1',	'Applied Scientific and clinical knowledge base (Row 6).1',	'Professional behaviour and ethical conduct (Row 7).1']
    competency_columns_student =  ['History, Examination & Investigations (Row 1)',	'Diagnosis/es Problem List (Row 2)',	'Treatment and Management Plan (Row 3)',	'Appropriate patient care and communication (Row 4)',	'Clinical and technical proficiency (Row 5)',	'Applied Scientific and clinical knowledge base (Row 6)',	'Professional behaviour and ethical conduct (Row 7)' ]
    competency_columns =  competency_columns_student + competency_columns_supervisor + ['Supervisor reflection', 'Overall Competency']
    # replace values in compentency columns to rename values 
    df[competency_columns] = df[competency_columns].replace(rename_values).astype(int, errors='ignore')
    # replace the not answered values with nan
    df[competency_columns] = df[competency_columns].replace('Not answered', np.nan)
    # display(df.head(5))
    filtered = df[[colId, colName, colDate] + procedure_columns + additional_procedure_columns + competency_columns]
    filtered = filtered.dropna(subset=[colId])
    # Remove non-numeric values from student ID
    filtered = filtered[filtered[colId].astype(str).str.isnumeric()]
    filtered[colId] = filtered[colId].astype(int)
    return filtered, procedure_columns, additional_procedure_columns

Filter out the important columns

In [6]:
def getProcedures(row, procedure_columns, additional_procedure_columns):
    procedures = []
    for col in procedure_columns:
        if not pd.isna(row[col]) and row[col] != 'Not answered':
            procedures.append(col)
    for col in additional_procedure_columns:
        if not pd.isna(row[col]) and row[col] != 'Not answered':
            procedures.append(str(row[col]))
    for item in procedures:
        # see if item is int convertible otherwise remove it
        try:
            int(item)
        except:
            procedures.remove(item)
    return procedures

In [ ]:
print(len(dfList))


overall_procedures = []
for username, dfList in dfListDict.items():
    overall_procedures = []
    filteredList = []
    elements = []
    name = dfList[0]['Student Name'].iloc[0]
    print(username, name)
    i=0
    for df in dfList:
        i+=1
        # print(df.columns)
        dfFiltered, procedure_columns, additional_procedure_columns = renameProcedureCols(df)
        # print(dfFiltered[colDate].dtype)
        # print(dfFiltered[colDate])
        dfFiltered[colDate] = pd.to_datetime(dfFiltered[colDate], format='mixed', errors='coerce').dt.date
        # print(dfFiltered.shape)
        # display(dfFiltered.head(3))
        filteredList.append(dfFiltered)
        
        # Add heading
        styles = getSampleStyleSheet()
        # Get the procedures list
        tableDict = {}
        for idx, row in dfFiltered.iterrows():
            # get the codes
            codes = getProcedures(row, procedure_columns, additional_procedure_columns)
            overall_procedures += codes
            print(idx, codes)
            codes_str = '\n'.join([code for code in codes])
            tableDict[row[colDate]] = codes_str

        # Transpose the tableDict to create a horizontal table
        dates = list(tableDict.keys())
        procedures = list(tableDict.values())
        # overall_procedures += procedures
        # Split the data into chunks that fit within the page width
        max_columns_per_row =  10 # Adjust this value based on the desired number of columns per row
        date_chunks = [dates[i:i + max_columns_per_row] for i in range(0, len(dates), max_columns_per_row)]
        procedure_chunks = [procedures[i:i + max_columns_per_row] for i in range(0, len(procedures), max_columns_per_row)]
        
        # Add the heading
        heading2Style = ParagraphStyle('Heading2', parent=styles['Heading2'], fontSize=28, alignment=1)  # Centered
        headingsuper = Paragraph(f"Rotation {i} procedures", heading2Style)
        elements.append(headingsuper)
        elements.append(Spacer(1, 32))

        # heading = Paragraph("Dates and Procedures Table", styles['Heading1'])
        # elements.append(heading)
        # elements.append(Spacer(1, 12))

        # Create the table for each chunk and add to elements
        for date_chunk, procedure_chunk in zip(date_chunks, procedure_chunks):
            transposed_data = [date_chunk, procedure_chunk]
            table = Table(transposed_data, colWidths=[1 * inch] * len(date_chunk))  # Adjust colWidths to fit the page
            table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),

                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ]))
            elements.append(table)
            elements.append(Spacer(1, 12))
        elements.append(PageBreak())

    # Convert the list of procedures to a pandas DataFrame
    df = pd.DataFrame(overall_procedures, columns=['Procedure'])

    # Count the occurrences of each procedure
    procedureCounts = df['Procedure'].value_counts()

    pageSize = ( 11.69 * inch, 8.27 * inch) # page size
    print(pageSize)
    figSize = (pageSize[0] / 100, pageSize[1] / 100)
    # Define the margins
    leftMargin = 0 * inch
    rightMargin = 0 * inch
    topMargin = 1 * inch
    bottomMargin = 0 * inch

    def addPlotImage(fig, ratio = None):
            plotImage = createPlotImage(fig)
            image = Image(plotImage)
            # print(image.drawWidth, image.drawHeight)
            
            # Resize image to fit within margins
            max_height = pageSize[1] - topMargin - bottomMargin  # Max height for Page
            max_width = pageSize[0] - leftMargin - rightMargin  # Max width for Page
            if ratio is not None:
                max_width = max_width * ratio
                max_height = max_height * ratio
            aspect_ratio = min(max_width / image.drawWidth, max_height / image.drawHeight)
            image.drawWidth *= aspect_ratio
            image.drawHeight *= aspect_ratio
            plt.close(fig)
            return image


    # Plot the bar chart
    fig, ax = plt.subplots(1, 1, figsize=(figSize[0]*1.1, figSize[1]))
    procedureCounts.plot(kind='bar', ax=ax)

    plt.title('Number of Procedures for each item')
    plt.xlabel('Procedures')
    plt.ylabel('Count')
    plt.xticks(rotation=90, fontsize=8)
    plt.tight_layout()

    # Save the figure to a file
    from io import BytesIO
    def createPlotImage(fig):
            buf = BytesIO()
            fig.savefig(buf, format='png', bbox_inches='tight')
            buf.seek(0)
            return buf

    image = addPlotImage(fig, 0.9)

    elements.append(image)
    # plt.close()
    doc = SimpleDocTemplate(f'{folderpath}/{name}.pdf', pagesize=pageSize, leftMargin=leftMargin, rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)

    # df.to_excel('BOH Third Year - ORAL30002/2024 ORAL30002Clinical Assessment Form All.xlsx', index=False)

In [ ]:
studentids = filtered[colId].unique()
pprint(studentids)
# create dataframe for each student
student_dfs = {}
for id in studentids:
    student_dfs[id] = (filtered[filtered[colId] == id])

Get Pairwise time series for each of the competency later

## Get the time series plot in image

In [ ]:
SupervisorlabelText = {4: 'Able to perform tasks independently', 3: 'Able to perform tasks under my indirect supervision', 
                       2: 'Able to perform tasks under my direct supervision', 1: 'Not yet able to perform tasks', 0: 'Not observed'}
StudentlabelText = {4: 'I am able to perform this task independently', 3:'I am able to perform this task with indirect/limited supervision', 
                 2: 'I am able to perform this task with direct/close supervision', 1: 'I am not yet able to perform this task', 0: 'I did not have to perform that task today'}

In [ ]:
def getProcedures(row):
    procedures = []
    for col in procedure_columns:
        if not pd.isna(row[col]) and row[col] != 'Not answered':
            procedures.append(col)
    for col in additional_procedure_columns:
        if not pd.isna(row[col]) and row[col] != 'Not answered':
            procedures.append(str(row[col]))
    for item in procedures:
        # see if item is int convertible otherwise remove it
        try:
            int(item)
        except:
            procedures.remove(item)
    return procedures

### In PDF format

In [ ]:
figSize = (8.27 * 2, 11.69)
nCompetencies = len(competency_columns_student)
def createMappingTable():
    data = [['Score', 'Student Label', 'Supervisor Label']]
    for i in range(5):
        data.append([i, StudentlabelText[i], SupervisorlabelText[i]])
    
    table = Table(data, colWidths=[1 * inch, 7 * inch, 7 * inch], rowHeights=1 * inch)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Vertically center text
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 24),  # Increased font size for the header row
        ('FONTSIZE', (0, 1), (-1, -1), 17),  # Increased font size for the data rows
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    return table


def plotTimeSeries(dummy, graphs_per_page, student_name, student_id, num_pages, page):
    plt.ioff()  # Turn off interactive mode to prevent showing plots while saving to PDF
    fig, axes = plt.subplots(graphs_per_page, 1, figsize=figSize, sharex=True)
    
    if graphs_per_page == 1:
        axes = [axes]  # Ensure axes is iterable when there's only one plot per page
 
    for i in range(graphs_per_page):
        comp_index = page * graphs_per_page + i
        if comp_index >= nCompetencies:
            axes[i].axis('off')  # Hide the empty subplot
            continue  # Skip the remaining steps for empty subplots
        
        compStudent = competency_columns_student[comp_index]
        compSupervisor = competency_columns_supervisor[comp_index]
        # print(compStudent, compSupervisor)
        data = dummy[[colDate, compStudent, compSupervisor] + procedure_columns + additional_procedure_columns]  
        # sort the data by date
        # convert date to datetime
        data[colDate] = pd.to_datetime(data[colDate], format='mixed', errors='coerce').dt.date
        data = data.sort_values(by=colDate)
        # print(data.columns)
        data[compStudent] = pd.to_numeric(data[compStudent], errors='coerce').astype('Int64')
        data[compSupervisor] = pd.to_numeric(data[compSupervisor], errors='coerce').astype('Int64')
        data.reset_index(drop=True, inplace=True)
        
        # Replace NA values with 0 for plotting
        data[compStudent].fillna(0, inplace=True)
        data[compSupervisor].fillna(0, inplace=True)
        
        ax = axes[i]
        
        # Set the width of the bars
        barWidth = 0.35

        # Set position of bar on X axis
        r1 = list(range(len(data[colDate])))
        r1 = [x * 1.5 for x in r1]
        r2 = [x + barWidth for x in r1]

        # Make the plot
        ax.bar(r1, data[compStudent], color='#00b3b3', width=barWidth, edgecolor='grey', label='Student')
        ax.bar(r2, data[compSupervisor], color='#ff6600', width=barWidth, edgecolor='grey', label='Supervisor')

        # Add xticks on the middle of the group bars
        ax.set_ylabel('Scores', fontweight='bold')
        ax.set_ylim(0, 6)
        ax.set_yticks([0, 1, 2, 3, 4])
        # ax.set_yticklabels([StudentlabelText[i] for i in range(5)]) # Set the y-tick labels to the student labels
        ax.legend()
        ax.set_title(f'{compStudent.split("(")[0]}') 
        ax.set_xticks([(r + barWidth / 2) for r in r1])
        ax.set_xticklabels(data[colDate].astype(str), rotation=90, fontsize=8)
        ax.tick_params(axis='both', which='both', labelsize=9, labelbottom=True)
        
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.subplots_adjust(top=0.88, hspace=0.6)  # Adjust the top spacing to add a gap between the suptitle and the subplots
    fig.suptitle(f'{student_name} ({student_id})', y=0.99)
    return fig, axes


fontSize = 16
import matplotlib.colors as mcolors

# Define a colorblind-friendly color palette
colorblind_palette = {
    1: '#E69F00',  # Label 1
    2: '#56B4E9',  # Label 2
    4: '#009E73',  # Label 3
    3: '#F0E442'   # Label 4
}

def plotPieCharts(data, student_name, student_id):
    # Create a figure for the overall pie charts
    figFull, axes = plt.subplots(1, 2, figsize=figSize)

    # Count the occurrences of scores in competency_columns_student and competency_columns_supervisor
    student_scores = data[competency_columns_student].stack().value_counts().sort_index()
    supervisor_scores = data[competency_columns_supervisor].stack().value_counts().sort_index()

    # Data for dataframe row addition
    addTodfRow = [
        student_id, 
        student_name, 
        supervisor_scores.get(4, 0), 
        supervisor_scores.get(3, 0), 
        supervisor_scores.get(2, 0), 
        supervisor_scores.get(1, 0)
    ]

    # Map student scores to corresponding colors from the dictionary
    student_colors = [colorblind_palette.get(i, '#999999') for i in student_scores.index]
    
    # Plot pie chart for student scores
    total_student_scores = student_scores.values.sum()
    axes[0].pie(student_scores, labels=student_scores.index.astype(int), autopct=lambda x: autopct(x, total_student_scores),
                startangle=140, textprops={'fontsize': fontSize}, colors=student_colors)
    axes[0].set_title('Overall Student Scores', fontsize=fontSize)

    # Map supervisor scores to corresponding colors from the dictionary
    supervisor_colors = [colorblind_palette.get(i, '#999999') for i in supervisor_scores.index]

    # Plot pie chart for supervisor scores
    total_supervisor_scores = supervisor_scores.values.sum()    
    axes[1].pie(supervisor_scores, labels=supervisor_scores.index.astype(int), autopct=lambda x: autopct(x, total_supervisor_scores), 
                startangle=140, textprops={'fontsize': fontSize}, colors=supervisor_colors)
    axes[1].set_title('Overall Supervisor Scores', fontsize=fontSize)

    figFull.suptitle(f'{student_name} ({student_id})', fontsize=fontSize*1.5)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    # Create a list of figures for individual competencies
    individual_figs = []
    
    competencyRow = {}
    for compStudent, compSupervisor in zip(competency_columns_student, competency_columns_supervisor):
        
        fig, ax = plt.subplots(1, 2, figsize=figSize)
        
        # Plot pie chart for individual competency (student)
        student_scores_individual = data[compStudent].value_counts().sort_index()
        student_colors_individual = [colorblind_palette.get(i, '#999999') for i in student_scores_individual.index]
        total = student_scores_individual.values.sum()
        ax[0].pie(student_scores_individual, labels=student_scores_individual.index.astype(int), 
                  autopct=lambda x: autopct(x, total), startangle=140, textprops={'fontsize': fontSize}, colors=student_colors_individual)
        ax[0].set_title(f'{compStudent.split("(")[0]}\n(Student)', fontsize=fontSize)
        
        # Plot pie chart for individual competency (supervisor)
        supervisor_scores_individual = data[compSupervisor].value_counts().sort_index()
        supervisor_colors_individual = [colorblind_palette.get(i, '#999999') for i in supervisor_scores_individual.index]
        competencyRow[compSupervisor] = [student_id, 
                                         student_name, 
                                         supervisor_scores_individual.get(4, 0), 
                                         supervisor_scores_individual.get(3, 0), 
                                         supervisor_scores_individual.get(2, 0),
                                         supervisor_scores_individual.get(1, 0)]

        total = supervisor_scores_individual.values.sum()
        ax[1].pie(supervisor_scores_individual, labels=supervisor_scores_individual.index.astype(int), 
                  autopct=lambda x: autopct(x, total), startangle=140, textprops={'fontsize': fontSize}, colors=supervisor_colors_individual)
        ax[1].set_title(f'{compSupervisor.split("(")[0]}\n(Supervisor)', fontsize=fontSize)
        
        # fig.suptitle(f'Student Name: {student_name} - Student ID: {student_id}', fontsize=fontSize*1.5)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        individual_figs.append(fig)
    
    return figFull, individual_figs, addTodfRow, competencyRow

In [ ]:
doneList =[]

In [ ]:
guttmanpath = 'FullSpreadsheets\\CAF+v0.1_September+15,+2024_23.24\\Fixed Pros DENT90120\\DENT90120 Fixed Pros guttman.xlsx'
# folder, filename, ext = getFolderandFileName(guttmanpath)
workbook, folder, filename = loadWorkbook(guttmanpath)
requiredItems = [541, 542, 543, 544, 545 ,551, 552, 553, 554, 555, 556, 572, 577, 611, 613, 615, 618,
                 625, 627, 631, 643, 651, 655]
lableCount = '# of Forms'
lengthDf1 = pd.DataFrame(columns=['Item', lableCount])
studentItemDf = pd.DataFrame(columns=['Item', 'Student ID'])
for sheet in workbook.sheetnames:
    code = sheet.split('_')[0]
    try:
        if not int(code) in requiredItems:
            print(f'{sheet} not in requiredItems')
            continue
    except ValueError:
        print(f'{sheet} not in requiredItems')
        continue
    df = loadDfFromSheet(workbook, sheet)
    # remove last row
    df = df.iloc[:-1]
    df[colId] = df[colId].astype(int)
    lengthDf1 = pd.concat([lengthDf1, pd.DataFrame({'Item': [code], lableCount: [df.shape[0]]})])
    studentItemDf = pd.concat([studentItemDf, pd.DataFrame({'Item': [code]*df.shape[0], 'Student ID': df['Student ID']})])

In [ ]:
# requiredItems = [541, 542, 543, 544, 545 ,551, 552, 553, 554, 555, 556, 572, 577, 611, 613, 615, 618,
#                  625, 627, 631, 643, 651, 655]
ItemCountDf = pd.DataFrame(columns=['Item', colId])
for student_id, dummy in student_dfs.items():
    for idx, row in dummy.iterrows():
        # get the codes
        codes = getProcedures(row)
        print(student_id, codes)    
        # remove codes that are not three digits
        codes = [code for code in codes if len(code) == 3 or 'la' in code.lower()]
        # replace items with la with '911'
        codes = ['911' if 'la' in code.lower() else code for code in codes]
        print(student_id, codes)    
        codeInt = [int(code) for code in codes]
        for code in codeInt:
            # if not code in requiredItems:
            #     continue
            ItemCountDf = pd.concat([ItemCountDf, pd.DataFrame({'Item': [code], colId: [student_id]})], axis=0)
# display(ItemCountDf.head())
lengthDf2 = ItemCountDf.groupby('Item').count().reset_index()
lengthDf2.rename({'Item': 'Item', colId: lableCount}, axis=1, inplace=True)
# display(lengthDf2)
lengthDf = pd.concat([lengthDf1, lengthDf2], axis=0).reset_index(drop=True)
# display(lengthDf)
lengthDf['Item'] = lengthDf['Item'].astype(int)
# group by item and count the number of forms
lengthDf = lengthDf.groupby('Item', as_index=False)['# of Forms'].sum()
lengthDf = lengthDf.sort_values(lableCount, ascending=False)
# display(lengthDf)
# plot the number of forms per item
# plt.figure(figsize=(15, 10))
# sns.barplot(x='Item', y='# of Forms', data=lengthDf, order=lengthDf['Item'])
# plt.title('Number of forms per Item')
# plt.savefig(f'{folder}/Number of forms per Item.png')
# plt.show()

studentItemCount = pd.concat([studentItemDf, ItemCountDf], axis=0).reset_index(drop=True)
studentItemCount['Item'] = studentItemCount['Item'].astype(int)
# display(studentItemCount)




In [ ]:
# student item df, aggregate over student id and add a column with count of items
originalStudentItemDf = studentItemCount.copy()
studentItemDfCount = originalStudentItemDf.groupby(['Student ID', 'Item']).size().reset_index(name='Count')
display(studentItemDfCount.head())
studentItemDfCount.to_excel(f'{folder}/StudentItemCounts.xlsx', index=False)
# For each Item, create a bar graph of student and Count
uniqueItems = studentItemDfCount['Item'].unique()
# for item in uniqueItems:
#     df = studentItemDfCount[studentItemDfCount['Item'] ==item]
#     df.sort_values('Count', ascending=False, inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     display(df.head())
#     plt.figure(figsize=(15, 10))
#     sns.barplot(x=colId, y='Count', data=df, order=df[colId])
#     plt.title(f'Number of Items for {item} per Student')
#     plt.xticks(rotation=90)
#     # y ticks only integers
#     plt.yticks(np.arange(0, df['Count'].max()+1, 1))
#     savefolder = f'{folder}/Student Counts'
#     os.makedirs(savefolder, exist_ok=True)
#     plt.savefig(f'{savefolder}/{item}.png')
#     plt.show()

# Get an overall count of items per student
studentCount = studentItemDfCount.groupby('Student ID').sum().reset_index()
studentCount.rename(columns={'Count': 'Total Count'}, inplace=True)
studentCount.sort_values('Total Count', ascending=False, inplace=True)
# display(studentCount.head())
# plt.figure(figsize=(35, 10))
# sns.barplot(x='Student ID', y='Total Count', data=studentCount, order=studentCount['Student ID'])
# plt.title('Total Number of Items per Student')  
# plt.xticks(rotation=90)
# plt.savefig(f'{folder}/Total Count.png')
# plt.show()

In [ ]:
# Take Student Item Df count, create columns for each item and fill with count, zero if not present
studentItemDfCount = studentItemDfCount.pivot(index='Student ID', columns='Item', values='Count').fillna(0).astype(int)
dds3students = pd.read_excel('2024 MDS Student List_v10.xlsx')
dds3students = dds3students[dds3students[colCohort] == 'DDS3 (2024)']
print(dds3students.shape)
# get the student ids in the dds3students that are not in the studentItemDfCount
missingStudents = dds3students[~dds3students[colId].isin(studentItemDfCount.index)]
# get the student ids in the studentItemDfCount that are not in the dds3students
extraStudents = studentItemDfCount[~studentItemDfCount.index.isin(dds3students[colId])]
display(missingStudents)
display(extraStudents)

studentItemDfCount = studentItemDfCount.reset_index()
# Add the missing students to the studentItemDfCount with all zeros
# Ensure that column names are strings
studentItemDfCount.columns = studentItemDfCount.columns.map(str)

# Now try concatenating
studentItemDfCount = pd.concat(
    [studentItemDfCount, 
     missingStudents[[colId]].assign(**{str(col): 0 for col in studentItemDfCount.columns[1:]})], 
    axis=0
)

# studentItemDfCount = pd.concat([studentItemDfCount, missingStudents[[colId]].assign(**{col: 0 for col in studentItemDfCount.columns[1:]})], axis=0)
display(studentItemDfCount.head())
# Rename the Item column to Student ID and reset index
# studentItemDfCount.rename_axis('Student ID').reset_index(inplace=True)
# save to excel with cells of 0 colored 'FFC7CE'
saveDf(studentItemDfCount, f'{folder}/{file} Student Item Counts.xlsx', 'Sheet1')

In [ ]:
# Assuming competency_columns_student, competency_columns_supervisor, colDate, studentids, student_dfs, procedure_columns, additional_procedure_columns, and getProcedures() are defined


nCompetencies = len(competency_columns_student)
graphs_per_page = 2  # Number of graphs to plot per page

overallScoresdf = pd.DataFrame(columns=['Student ID', 'Student Name', '4', '3', '2', '1'])
competencydf = {}
for i in range(nCompetencies):
    competencydf[competency_columns_supervisor[i]] = pd.DataFrame(columns=['Student ID', 'Student Name', '4', '3', '2', '1'])

for student_id, dummy in student_dfs.items():
    if student_id in doneList:
        continue
    student_name = dummy[colName].iloc[0]
    pdf_filename = os.path.join(output_dir, f'Competency_{student_id}.pdf')
    with PdfPages(pdf_filename) as pdf:
        num_pages = (nCompetencies + graphs_per_page - 1) // graphs_per_page  # Calculate number of pages needed

        # Plot and save time series
        for page in range(num_pages):
            fig, axes = plotTimeSeries(dummy, graphs_per_page, student_name, student_id, num_pages, page)
            # Save the current figure to the PDF as a new page
            pdf.savefig(fig)
            plt.close(fig)
        
        # Plot and save pie charts
        overall_pie_fig, individual_pie_figs, addTodfRow, competencyRow = plotPieCharts(dummy, student_name, student_id)
        pdf.savefig(overall_pie_fig)
        plt.close(overall_pie_fig)
        
        for pie_fig in individual_pie_figs:
            pdf.savefig(pie_fig)
            plt.close(pie_fig)
    
    # add the data to df
    overallScoresdf.loc[len(overallScoresdf)] = addTodfRow
    for key, value in competencyRow.items():
        competencydf[key].loc[len(competencydf[key])] = value
        
    # Get the procedures list
    tableDict = {}
    for idx, row in dummy.iterrows():
        # get the codes
        codes = getProcedures(row)
        codes_str = '\n'.join([code for code in codes])
        tableDict[row[colDate]] = codes_str

    # Transpose the tableDict to create a horizontal table
    dates = list(tableDict.keys())
    procedures = list(tableDict.values())
    
    # Split the data into chunks that fit within the page width
    max_columns_per_row = 8  # Adjust this value based on the desired number of columns per row
    date_chunks = [dates[i:i + max_columns_per_row] for i in range(0, len(dates), max_columns_per_row)]
    procedure_chunks = [procedures[i:i + max_columns_per_row] for i in range(0, len(procedures), max_columns_per_row)]

    # Save tableDict to the PDF
    table_pdf_filename = pdf_filename.replace(".pdf", "_table.pdf")
    doc = SimpleDocTemplate(table_pdf_filename, pagesize=(8.27 * inch * 2, 11.69 * inch))
    elements = []

    # Add heading
    styles = getSampleStyleSheet()
    heading = Paragraph("Dates and Procedures Table", styles['Heading1'])
    elements.append(heading)
    elements.append(Spacer(1, 12))

    # Create the table for each chunk and add to elements
    for date_chunk, procedure_chunk in zip(date_chunks, procedure_chunks):
        transposed_data = [date_chunk, procedure_chunk]
        table = Table(transposed_data, colWidths=[1.65 * inch] * len(date_chunk))  # Adjust colWidths to fit the page
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),

            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ]))
        elements.append(table)
        elements.append(Spacer(1, 12))

    doc.build(elements)

    # Create the mapping table and add it to the first page
    mappingTable = createMappingTable()
    
    # Create a temporary PDF to hold the table
    temp_pdf_filename = 'temp_mapping_table.pdf'
    doc = SimpleDocTemplate(temp_pdf_filename, pagesize=(8.27 * inch * 2, 11.69 * inch))
    elements = []
    
    # Custom styles
    styles = getSampleStyleSheet()
    heading_style = ParagraphStyle('Heading1', parent=styles['Heading1'], fontSize=32, alignment=1)  # Centered
    subheading_style = ParagraphStyle('Heading2', parent=styles['Heading2'], fontSize=28, alignment=1)  # Centered
    
    elements.append(Paragraph(f'{student_name} ({student_id})', heading_style))
    elements.append(Spacer(1, 24))
    elements.append(Paragraph("Mapping Table", subheading_style))
    elements.append(Spacer(1, 36))
    elements.append(mappingTable)
    doc.build(elements)
    


    # Merge the plot PDF and table PDF into one file
    output = PdfWriter()

    # Add the mapping table to the PDF
    with open(temp_pdf_filename, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            output.add_page(page)
    
    # Clean up the temporary PDF file
    os.remove(temp_pdf_filename)
    
    # Read the original plots PDF
    with open(pdf_filename, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            output.add_page(page)

    # Read the table PDF
    with open(table_pdf_filename, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            output.add_page(page)

    # Write the merged PDF
    with open(pdf_filename, "wb") as f:
        output.write(f)
    
    # Cleanup the intermediate table PDF
    os.remove(table_pdf_filename)
    doneList.append(student_id)

    # break  # Remove this line to generate all competencies in one PDF file

In [ ]:
overallScoresdf.head()
overallScoresdf.to_excel(f'{output_dir}\\OverallScores.xlsx', index=False)

In [ ]:

def plotStackedBarChart(df, title = '', savepath = ''):
    # Ensure 'Student ID' is treated as a string
    df['Student ID'] = df['Student ID'].astype(str)

    # Set 'Student ID' as the index for plotting
    df.set_index('Student ID', inplace=True)

    # Plot the stacked bar chart
    ax = df[['1', '2', '3', '4']].plot(kind='bar', stacked=True, figsize=(16, 8), color=['#FF0099', '#66B2FF', '#99FF99', '#FFCC99'])
    
    # Set labels and title
    ax.set_xlabel('Student ID')
    ax.set_ylabel('Scores')
    ax.set_title(f'Counts of Student Scores{title}')

    # Add a legend
    # Add a legend with custom labels
    handles, labels = ax.get_legend_handles_labels()
    new_labels = [SupervisorlabelText[int(label)] for label in labels]
    ax.legend(handles, new_labels, title='Scores', bbox_to_anchor=(0.75, 1.25), loc='upper left')
    
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=80)
    
    # Show plot
    plt.tight_layout()
    plt.savefig(savepath)
    plt.show()
    
plotStackedBarChart(overallScoresdf.copy(), '', f'{output_dir}\\OverallScores.png')

In [ ]:
for key, df in competencydf.items():
    # df.to_excel(f'BOH Third Year - ORAL30002\\FullPlots\\{key}.xlsx', index=False)
    title = f' for {key.split("(")[0]}'
    filename = key.split("(")[0]
    # remove / from the filename
    filename = filename.replace('/', '-')
    savepath = f'{output_dir}\\{filename}.png'
    plotStackedBarChart(df.copy(), title, savepath)
    

## Trash

In [ ]:
# nCompetencies = len(competency_columns_student)
# dummy = student_dfs[studentids[0]]
# # dummy = student_dfs[1337581]
# for i in range(nCompetencies):
#     compStudent = competency_columns_student[i]
#     compSupervisor = competency_columns_supervisor[i]
#     data = dummy[[colDate, compStudent, compSupervisor] + procedure_columns + additional_procedure_columns]
#     data[compStudent] = pd.to_numeric(data[compStudent], errors='coerce').astype('Int64')
#     data[compSupervisor] = pd.to_numeric(data[compSupervisor], errors='coerce').astype('Int64')
#     data.reset_index(drop=True, inplace=True)
#     # pprint(data)
#     # Drop rows with NaN values in competencies
#     # data.dropna(subset=[compStudent, compSupervisor], inplace=True)

        
#     # Replace NA values with 0 for plotting
#     data[compStudent].fillna(0, inplace=True)
#     data[compSupervisor].fillna(0, inplace=True)
    
#     # Plotting
#     fig, ax = plt.subplots(figsize=(18, 6))
    
#     # Set the width of the bars
#     barWidth = 0.35


#     # Set position of bar on X axis
#     r1 = range(len(data[colDate]))
#     r2 = [x + barWidth for x in r1]
#     print(r1)
#     # Make the plot

#     ax.bar(r1, data[compStudent], color='#00b3b3', width=barWidth, edgecolor='grey', label='Student')
#     ax.bar(r2, data[compSupervisor], color='#ff6600', width=barWidth, edgecolor='grey', label='Supervisor')

#     # Annotate bars with 'Not Assessed' for NaN values
#     for idx, row in data.iterrows():
#         # get the codes
#         codes = getProcedures(row)
#         # print(row[colDate],  codes)
#         codes_str = '\n'.join([code for code in codes])
#         pos = (r1[idx] + r2[idx])/2

#         # ax.text(pos, 4.5, codes_str, ha='center', va='center', color='black')

#         # if row[compStudent]==0:
#         #     # print(idx)
#         #     ax.annotate('Not Observed', (r1[idx], 2.5), rotation=90, ha='center', va='center', color='black', fontsize=8)
#         # if (row[compSupervisor]==0):
#         #     ax.text(r2[idx], 2.5, 'Not Observed', rotation=90, ha='center', va='center', color='black', fontsize=8)
#     # ax.plot(r1, data[compStudent], color='b', marker='o', label='Student')
#     # ax.plot(r2, data[compSupervisor], color='r', marker='o')
    
#     # Add xticks on the middle of the group bars
#     ax.set_xlabel(colDate, fontweight='bold')
#     ax.set_xticks([r + barWidth/2 for r in range(len(data[colDate]))])
#     ax.set_xticklabels(data[colDate].astype(str), rotation=90)
    
#     ax.set_ylabel('Scores', fontweight='bold')
#     ax.set_ylim(0, 5)
#     ax.set_yticks([0, 1, 2, 3, 4])

#     # Create legend & Show graphic
#     plt.legend()
#     plt.title(f'Competency: {compStudent.split("(")[0]} for Student ID: {id}') 
#     plt.tight_layout()
#     plt.show()


## For each code get cumulative data

In [ ]:
# age_columns = [col for col in df.columns if 'Patient Age' in col]
# print(age_columns)
# procedure_columns.sort()
# print(procedure_columns)
# print(additional_procedure_columns)
# student_id = df['Student ID'].unique()
# studentidname_dict = {}
# for student in student_id:
#     studentidname_dict[student] = df[df['Student ID'] == student]['Respondent'].values[0]
# print(studentidname_dict)
# print(student_id)
# # create different dataframe for each student
# student_dfs = {}

# for student in student_id:
#     student_dfs[student] = df[df['Student ID'] == student]
# student_dfs['All'] = df.copy()

# # Define age groups
# age_groups = {
#     'under 6': (0, 5),
#     '6 to 12': (6, 12),
#     '13 to 18': (13, 18),
#     'Over 18': (19, 200)
# }

# # Initialize results dictionary


In [ ]:
# def get_results(df):
#     results = {
#         'under 6': {'count': 0, 'procedures': [], 'scores': {}},
#         '6 to 12': {'count': 0, 'procedures': [], 'scores': {}},
#         '13 to 18': {'count': 0, 'procedures': [], 'scores': {}},
#         'Over 18': {'count': 0, 'procedures': [], 'scores': {}}
#     }

#     for _, row in df.iterrows():
#         for age_col in age_columns:
#             try:
#                 age = float(row[age_col])
#             except (ValueError, TypeError):
#                 continue
            
#             if pd.notnull(age):
#                 for group, (min_age, max_age) in age_groups.items():
#                     if min_age <= age <= max_age:
#                         results[group]['count'] += 1
#                         procedures = []

#                         for proc_col in procedure_columns:
#                             if pd.isna(row[proc_col]) or row[proc_col] == 'nan':  # Use pd.isna() to check for NaN values
#                                 continue
#                             if row[proc_col] == "Yes":
#                                 procedures.append(proc_col)
#                             if row[proc_col] not in ['Not answered', 'Yes']:
#                                 procedures.append(row[proc_col])
                        
#                         # Collect scores for the age group
#                         for i, col in enumerate(competency_columns_supervisor[:-2]):
#                             if col not in results[group]['scores'].keys():
#                                 results[group]['scores'][col] = []
#                             if not pd.isna(row[col]):
#                                 results[group]['scores'][col].append(row[col])
#                             elif not pd.isna(row[competency_columns_student[i]]):
#                                 results[group]['scores'][col].append(row[competency_columns_student[i]])

#                         # Collect procedures for the age group
#                         results[group]['procedures'].extend(procedures)
    
#     return results

# student_data = {}
# for student, student_df in student_dfs.items():
#     results = get_results(student_df)
#     student_data[student] = results

# student_data[1353787]

                    

In [ ]:
# tmp_dir = 'BOH Third Year - ORAL30002/tmp'
# os.makedirs(tmp_dir, exist_ok=True)

# def plot_results(data, title):
#     # Create a DataFrame from the data
#     df = pd.DataFrame(data).T
    
#     # Create count bar plot
#     plt.figure(figsize=(10, 6))
#     plt.bar(df.index, df['count'])
#     plt.xlabel('Age Group')
#     plt.ylabel('Count')
#     plt.ylim(0, max(df['count'].max()*1.1, df['count'].max() + 5))
#     plt.title('Count of Procedures by Age Group {}'.format(title))
#     for idx, value in enumerate(df['count']):
#         plt.text(idx, value, str(value), ha='center', va='bottom')
#     plt.tight_layout()
#     plt.text(0.90, 0.95, f'Total: {df["count"].sum()}', transform=plt.gca().transAxes, fontsize=12,
#              verticalalignment='center', bbox=dict(facecolor='white', edgecolor='black'))
#     plt.savefig(f'{tmp_dir}/temp_count_plot.png')  # Save figure to file
#     plt.close()
    
#     # Create subplots for procedures
#     fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18, 10))
#     axes = axes.flatten()
#     for i, (ageGroup, groupData) in enumerate(data.items()):
#         procedureCounts = {proc: 0 for proc in procedure_columns}
#         for proc in groupData['procedures']:
#             if proc in procedureCounts:
#                 procedureCounts[proc] += 1

#         # Sort procedures by count in descending order
#         procNames = list(procedureCounts.keys())
#         procValues = list(procedureCounts.values())
#         sortedProcs = sorted(zip(procNames, procValues), key=lambda x: x[1], reverse=True)
#         # remove the procedures with 0 count
#         sortedProcs = [(proc, count) for proc, count in sortedProcs if count > 0]

#         # If no procedures, skip this plot
#         if not sortedProcs:
#             axes[i].set_title(f'Procedures for {ageGroup.replace("_", " ").title()}')
#             axes[i].text(0.5, 0.5, 'No procedures', horizontalalignment='center', verticalalignment='center', transform=axes[i].transAxes)
#             axes[i].axis('off')
#             continue

#         procNames, procValues = zip(*sortedProcs)

#         axes[i].bar(procNames, procValues)
#         axes[i].set_title(f'Procedures for {ageGroup.replace("_", " ").title()}')
#         axes[i].set_xlabel('Procedure')
#         axes[i].set_ylabel('Count')
#         axes[i].tick_params(axis='x', rotation=90)
#         for j, value in enumerate(procValues):
#             if value > 0:
#                 axes[i].text(j, value, str(value), ha='center', va='bottom')

#         # Add total count box on the right side
#         total_procs = sum(procValues)
#         axes[i].text(0.87, 0.90, f'Total: {total_procs}', transform=axes[i].transAxes, fontsize=12,
#                      verticalalignment='center', bbox=dict(facecolor='white', edgecolor='black'))
    
#     plt.suptitle('Procedures by Age Group {}'.format(title), fontsize=16)
#     plt.tight_layout()
#     fig.savefig(f'{tmp_dir}/temp_procedures_plot.png')  # Save figure to file
#     plt.close(fig)


In [ ]:
# from pprint import pprint
# id = list(student_dfs.keys())[0]
# print(id)
# data = student_data[id]
# # pprint(data)
# plot_results(data, f'for {id}')

In [ ]:
# from matplotlib.backends.backend_pdf import PdfPages
# with PdfPages('BOH Third Year - ORAL30002/student_procedures_report.pdf') as pdf:
#  for studentId, data in student_data.items():
#         studentName = studentidname_dict.get(studentId, '')
#         title = f'for {studentId} - {studentName}' if studentId != 'All' else 'All Students'
        
#         # Create a new figure for each student to ensure they start on a new page
#         fig = plt.figure(figsize=(10, 6))
#         plot_results(data, title)
        
#         # Load saved figures
#         count_plot = plt.imread(f'{tmp_dir}/temp_count_plot.png')
#         procedures_plot = plt.imread(f'{tmp_dir}/temp_procedures_plot.png')

#         # Create a new figure for the combined plots
#         fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18, 14))
#         if studentId != 'All':
#                 fig.suptitle(f'Student ID: {studentId} - Name: {studentName}', fontsize=16)
#         else:
#                 fig.suptitle('All Students', fontsize=16)
#         axes[0].imshow(count_plot)
#         axes[0].axis('off')
#         axes[1].imshow(procedures_plot)
#         axes[1].axis('off')
        
#         plt.suptitle(f'{studentId} - {studentName}', fontsize=16)
#         plt.tight_layout()
        
#         # Save the current figure to the PDF
#         pdf.savefig(fig)
#         plt.close(fig)

# print('PDF report has been generated.')